In [1]:
import os
import sys
import pickle
import json
import pandas as pd
import numpy as np
from typing import Optional
from src.utils.databases import Database, Person, MailBasedDatabase
from src.main import load_main_db, get_userlist_file_from_dynamics, get_riegenlist_file_from_dynamics, path, ADULT_CAT, is_jugend_riege, get_riegen

In [3]:
userlist_file = get_userlist_file_from_dynamics(path)
riegenlist_file = get_riegenlist_file_from_dynamics(path)

InvalidSessionIdException: Message: WebDriver session does not exist, or is not active
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
InvalidSessionIDError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:448:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:515:13
assert.session@chrome://remote/content/shared/webdriver/Assert.sys.mjs:37:4
despatch@chrome://remote/content/marionette/server.sys.mjs:315:19
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20


In [64]:
userlist_file = 'MitgliederExportList_141024183131.xlsx'
riegenlist_file = 'OrgAdrList_1.01.02.5436.00-0_20241014_1831.csv'

In [65]:
main_db = load_main_db(path + userlist_file, path + riegenlist_file)

/Users/Lukas/Documents/Programming/Git/STVadmin_processing/.venv/lib/python3.12/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [66]:
spola_df = pd.read_excel("/Users/Lukas/Library/CloudStorage/OneDrive-FreigegebeneBibliotheken–TurnvereinWürenlos/Spola - Dokumente/Kinder/TN Liste.xlsx")

In [67]:
def str_contains(string, candidate):
    return candidate in string

def match_tn_to_db(row):
    geb = row["Geburtsdatum"]
    vorname = row["Vorname"]
    nachname = row["Nachname"]
    same_birthday = main_db.lookup_by_property("birthday", geb)
    same_fname = main_db.lookup_by_property("first_name", vorname, str_contains)
    same_lname = main_db.lookup_by_property("last_name", nachname)
    
    combined = set.intersection(set(same_birthday), set(same_fname),set(same_lname))
    if len(combined) == 1:
        return 1
    return 0

In [68]:
spola_df["Mitglied (Ja/Nein)"] = spola_df.apply(match_tn_to_db, axis=1)

In [69]:
spola_df["Mitglied (Ja/Nein)"].values


array([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1])

In [85]:
for person in main_db.lookup_by_property("last_name", "Geissmann"):
    print(f"{person.first_name} {person.last_name}, {person.birthday}")

Lynn Geissmann, 2010-01-07 00:00:00
Yann Geissmann, 2012-06-18 00:00:00
